# VAns—可变结构电路

Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.

## Overview

变分量子算法 (Variational Quantum Algorithm, VQA) 是一种使用经典优化器来训练参数化量子电路从而最小化特定的损失函数的算法。常见的变分量子算法，例如变分量子本征求解器（Variational Quantum Eigensolvers, VQE）和量子近似优化算法（Quantum Approximate Optimization Algorithm, QAOA），需要一个预设的固定参数化量子电路来完成优化过程。当电路过于简单时，其表达能力不够强，不足以得到目标损失函数的最优值。另一方面，如果电路过于复杂，虽然其表达能力很强，但是会受到贫瘠高原现象（Barren Plateau, BP）的影响，梯度的消失会使得优化器无法得到全局最优的解。所以，我们需要选择合适的电路结构来解决特定的问题，而一个好的电路设计算法就可以帮助我们找到这种电路。

在本教程中，我们将介绍一种叫做 VAns 的可变结构电路算法 [1]，这种算法可以针对特定的问题找到一个较浅的电路完成优化过程。从一个简单的初始电路开始，VAns 算法在优化的过程中会不断向电路中添加和删减量子门模块，从而在优化损失函数的同时也能保持较短的电路的深度。我们将以量子本征求解这个问题作为例子来展示 VAns 算法，并和原来使用固定电路结构的 VQE 算法进行对比。

## 算法流程

VAns 算法由如下步骤组成：
1. 准备一个简单的初始电路，使用经典优化器对电路参数进行调整，最小化目标损失函数，记录最优值。
2. 从一个集合中随机选择量子门模块（如下图所示，各量子门模块仅由 $R_y$ 门、$R_z$ 门，和 $CNOT$ 门组成），随机选择插入模块所作用的量子比特，并将模块添加到电路末尾。添加模块中的量子门的参数都初始化为 $0$ ，这样添加的模块一开始等同于 $I$。

![Inserting Blocks](./figures/vans-fig-blocks.png)

3. 根据下图中的规则化简电路，对化简后的电路进行优化，更新其参数，获得损失函数的当前最优值。比较当前损失函数最优值与上一个记录的最优值，根据设定的阈值决定是否接受新电路。若接受新电路，则继续遍历电路中的量子门，并删除不会降低损失的门，将精简过的电路记为当前电路，并记录其最优损失。若拒绝则直接回到第2步。

![Simplification rules](./figures/vans-fig-rules.png)

4. 重复步骤2-3，达到设定的迭代次数后停止，输出电路和最优损失。

## 量桨实现

我们以求解氢分子的基态能量为例来展示如何在量桨上使用 VAns 算法，这里我们着重展示如何优化和精简电路结构，优化损失函数的过程同[变分量子本征求解器（VQE）](https://qml.baidu.com/tutorials/quantum-simulation/variational-quantum-eigensolver.html)。我们通过下面几行代码引入必要的包。

In [1]:
import paddle
import paddle_quantum
import paddle_quantum.qchem as qchem
import numpy as np
from paddle_quantum.ansatz import Circuit
from paddle_quantum.ansatz.vans import Inserter, Simplifier, VAns, cir_decompose
from paddle_quantum.hamiltonian import Hamiltonian
import warnings

warnings.filterwarnings("ignore")
np.random.seed(11)
paddle.seed(11)

我们首先需要构造氢分子的哈密顿量，具体的说明可见[变分量子本征求解器（VQE）](https://qml.baidu.com/tutorials/quantum-simulation/variational-quantum-eigensolver.html)。

In [ ]:
geo = qchem.geometry(structure=[["H", [-0.0, 0.0, 0.0]], ["H", [-0.0, 0.0, 0.74]]])
# 将分子信息存储在 molecule 里，包括单体积分（one-body integrations），双体积分（two-body integrations），分子的哈密顿量等
molecule = qchem.get_molecular_data(
    geometry=geo,
    basis="sto-3g",
    charge=0,
    multiplicity=1,
    method="fci",
    if_save=True,
    if_print=True,
)
# 提取哈密顿量
molecular_hamiltonian = qchem.spin_hamiltonian(
    molecule=molecule,
    filename=None,
    multiplicity=1,
    mapping_method="jordan_wigner",
)
# n 为量子比特数
n = molecular_hamiltonian.n_qubits

接下来我们需要定义损失函数，也就是电路输出量子态关于该哈密顿量的期望值。

In [3]:
# 定义损失函数
def loss_func(cir: Circuit, H: Hamiltonian) -> paddle.Tensor:
    return cir().expec_val(H)

在训练前，我们还需要设定一些有关 VAns 算法和优化器的参数。读者可以自行调整这些参数来观察 VAns 算法的变化。

In [4]:
EPSI = 0.001 # 插入模块初始化参数的浮动范围
IR = 1 # 添加模块的速率
ITERI = 120 # 参数优化的迭代次数
ITERO = 5 # 结构优化的迭代次数
LR = 0.1 # 学习率
T = 0.01 # 删除量子门时允许损失值上升的范围
A = 100 # 对于更新电路的采纳率
IS0 = True # 如果初始态为 |0>, 则设为 True

paddle_quantum.set_backend('state_vector') # 设置运行模式为态矢量模式

In [5]:
# 初始化一个 VAns 的模块对象
vans = VAns(n, loss_func, molecular_hamiltonian,
           epsilon=EPSI,
           insert_rate=IR,
           iter=ITERI,
           iter_out=ITERO,
           LR=LR,
           threshold=T,
           accept_wall=A,
           zero_init_state=IS0)

### 初始电路

我们给出了一个简单的初始电路，其参数是从均匀分布中随机选取的。对于这个初始电路，我们运行优化过程使损失函数最小化。优化过程和普通的 VQE 算法相同，参数 **iter** 和 **LR** 分别决定了优化器迭代次数和学习率。记录优化后的参数化电路以及损失，将其作为结构优化的起始点。

In [6]:
# 优化初始电路
itr_loss = vans.optimization(vans.cir) 

# 更新 vans 的损失值
vans.loss = itr_loss

# 打印当前电路
print("当前电路:\n" + str(vans.cir))

iter: 20 loss: [-0.99608546]
iter: 40 loss: [-1.0926807]
iter: 60 loss: [-1.1136395]
iter: 80 loss: [-1.1163319]
iter: 100 loss: [-1.1167175]
iter: 120 loss: [-1.1167547]
当前电路:
--Rx(3.144)----Rz(3.512)----*----Rx(6.286)----Rz(2.441)-------------------------------------------------------------x--
                            |                                                                                       |  
----------------------------x----Rx(4.249)----Rz(3.141)----Rx(4.246)----Rz(5.477)----*------------------------------|--
                                                                                     |                              |  
--Rx(0.001)----Rz(5.499)----*--------------------------------------------------------x----Rx(3.141)----Rz(2.688)----|--
                            |                                                                                       |  
----------------------------x----Rx(0.003)----Rz(2.362)----Rx(0.003)----Rz(1.500)----------------------

### 插入量子门模块

现在我们从量子门模块集合中随机选取模块，随机选取插入模块所作用的量子比特，并将模块插入到当前电路的尾端。注意新插入模块中的量子门的参数都被设置为 $0$，这样添加模块前后的电路实际上是相同的，也就是说更新后的电路可以继承之前电路的损失。下面的代码展示了如何在电路中插入模块。一次插入模块的数量由参数 **insert_rate** 决定。另一个参数 **epsilon** 是用来设定插入模块初始参数与 $0$ 之间的差别。

In [7]:
# 向电路中添加模块，在这么做之前需要将电路从 layer 分解为量子门
new_cir = cir_decompose(vans.cir)
new_cir = Inserter.insert_identities(new_cir, vans.insert_rate, vans.epsilon)

# 打印更新过后的电路
print("添加过后的电路:\n" + str(new_cir))

添加过后的电路:
--Rx(3.144)----Rz(3.512)----*----Rx(6.286)----Rz(2.441)----------------------------------------------------------------------------------------------------x--
                            |                                                                                                                              |  
----------------------------x----Rz(0.000)----Rx(0.000)----Rx(-0.00)----Rx(4.249)----Rz(3.141)----Rx(4.246)----Rz(5.477)----*------------------------------|--
                                                                                                                            |                              |  
--Rx(0.001)----Rz(5.499)----*-----------------------------------------------------------------------------------------------x----Rx(3.141)----Rz(2.688)----|--
                            |                                                                                                                              |  
----------------------------x----Rx(0

### 简化电路

接下来我们需要化简当前的量子电路。化简电路的规则十分简单：
1. 将连续的 $CNOT$ 门合并消除；
2. 将连续的旋转门合并起来；
3. 若电路初始态为 $|0\rangle$，删除电路前端的 $CNOT$ 门 和 $Rz$ 门；
4. 交换旋转门和 $CNOT$ 门的位置以便进一步化简。

化简电路后，我们对电路中的参数进行优化以最小化损失函数，得到更新参数后的电路和对应的损失值。如果新的损失值小于之前记录的损失值，我们就接受这个电路作为新电路。如果新损失值大于之前的损失值，那么我们以一定概率接受这个电路，接受的概率与参数 **accept_wall** 有关。为了进一步简化电路，在接受新电路后，我们还会遍历电路中的量子门，如果删除某个量子门后，损失不会降低或者只会提高一个小于 **threshold** 的值，那么我们将该量子门删除，否则将其保留。

In [8]:
# 根据简化规则简化电路
new_cir = Simplifier.simplify_circuit(new_cir, vans.zero_init_state)

# 打印简化过后的电路
print(new_cir)

# 对简化过后的电路进行优化
itr_loss = vans.optimization(new_cir)

# 计算损失值的变化
relative_diff = (itr_loss - vans.loss) / np.abs(itr_loss)

# 若损失值降低或升高的幅度不大于设定的阈值，那么就接受电路
if relative_diff <= 0 or np.random.random() <= np.exp(
    -relative_diff * vans.accept_wall
):
    print("Accpet the new circuit!")

    # 移除不会降低损失的量子门
    new_cir = vans.delete_gates(new_cir, itr_loss)
    new_cir = Simplifier.simplify_circuit(new_cir, vans.zero_init_state)
    itr_loss = loss_func(new_cir, *vans.loss_args)
    vans.loss = itr_loss
else:
    print("Decline the new circuit!")

--Rx(3.144)----Rz(3.512)----*----Rx(6.286)----Rz(2.441)------------------------------------------------x--
                            |                                                                          |  
----------------------------x----Rz(4.491)----Rx(4.470)----Rz(8.712)----*------------------------------|--
                                                                        |                              |  
--Rx(0.001)----Rz(5.499)----*-------------------------------------------x----Rx(3.141)----Rz(2.688)----|--
                            |                                                                          |  
----------------------------x----Rz(3.604)----Rx(5.126)----Rz(4.462)-----------------------------------*--
                                                                                                          
iter: 20 loss: [-1.1121466]
iter: 40 loss: [-1.1123195]
iter: 60 loss: [-1.1158162]
iter: 80 loss: [-1.1166465]
iter: 100 loss: [-1.1167465]
ite

简化后的电路如下所示。

In [9]:
# 更新当前电路
vans.cir = new_cir

print("当前电路为:\n" + str(vans.cir))

当前电路为:
--Rx(3.142)----*----------------------x--
               |                      |  
---------------x----*-----------------|--
                    |                 |  
--------------------x----Rx(3.142)----|--
                                      |  
--------------------------------------*--
                                         


我们将上述的插入量子门模块和简化电路的步骤一起作为电路结构优化的一次迭代，总的迭代次数由参数 **iter_out** 决定。

### 简化版本

上述过程清晰明了地展示了 VAns 的运行步骤和原理，包括了量子门模块添加以及电路简化的过程，然而实际在量桨中使用 VAns 可以不必了解这些繁杂的步骤，因为量桨提供了一个经过封装的 VAns 算法，以便用户进行调用。下面的一行代码可以完成所有的电路结构优化以及参数训练过程。

In [10]:
# 使用 VAns 中内置的 train() 函数直接完成整个训练过程
vans.train()

Out iteration 1 for structure optimization:
iter: 20 loss: [-1.1167215]
iter: 40 loss: [-1.1166948]
iter: 60 loss: [-1.1167493]
iter: 80 loss: [-1.116759]
iter: 100 loss: [-1.1167591]
iter: 120 loss: [-1.1167595]
 Current loss: [-1.1167595]
 Current cir:
--Rx(3.142)----*----------------------x--
               |                      |  
---------------x----*-----------------|--
                    |                 |  
--------------------x----Rx(3.142)----|--
                                      |  
--------------------------------------*--
                                          

Out iteration 2 for structure optimization:
iter: 20 loss: [-1.0998794]
iter: 40 loss: [-1.1144476]
iter: 60 loss: [-1.1166729]
iter: 80 loss: [-1.1167666]
iter: 100 loss: [-1.1207738]
iter: 120 loss: [-1.1371526]
     accpet the new circuit!
     start deleting gates
         Deletion: reject deletion
         Deletion: accept deletion with acceptable loss
         Deletion: reject deletion
         Del

最终 VAns 算法给出的电路如下：

In [11]:
print("最终电路:\n" + str(vans.cir))
print("最终损失:\n" + str(vans.loss))

最终电路:
--Rx(3.141)----*------------------------------------------------------------------x--
               |                                                                  |  
---------------x----*----Rx(-0.22)----Rz(1.497)----*--------*---------------------|--
                    |                              |        |                     |  
--------------------|------------------------------|--------x--------Rx(3.141)----|--
                    |                              |                              |  
--------------------x------------------------------x----Rz(3.068)-----------------*--
                                                                                     
最终损失:
-1.1372840404510498


## 与原始 VQE 的对比

通过上面的结果我们不难发现，通过 VAns 得到的电路仅含有5个参数，电路的深度为9，得到的最小损失值为 $-1.13728392$ Ha。而原始的 VQE 算法（见[变分量子本征求解器（VQE）](https://qml.baidu.com/tutorials/quantum-simulation/variational-quantum-eigensolver.html)）则需要一个含12个参数且深度为11的电路，由此可见 VAns 可以极大的减少电路中的参数数量并保持较浅的的电路深度。

_______

## 参考文献

[1] Bilkis, M., et al. "A semi-agnostic ansatz with variable structure for quantum machine learning." [arXiv preprint arXiv:2103.06712 (2021).](https://arxiv.org/abs/2103.06712)